In [ ]:
import pandas as pd

# Load the user's CSV file to examine its structure and content
file_path = '/mnt/data/phishing.csv'
phishing_data = pd.read_csv(file_path)

# Display the first few rows to understand the dataset structure
phishing_data.head()